# Preparing the Data for Training

### This Jupyter Notebook replicated the preprocessing steps on the original paper for better understanding

The architecture of DeepSignalingFlow forces us to create seperate networks for each available dataset depending on the number of genes and drug combinations. We will thus start by only preprocessing the NCI ALMANAC Dataset.

To prepare we need to download the following Datasets:

NCI ALMANAC [Download](https://wiki.nci.nih.gov/spaces/NCIDTPdata/pages/338237347/NCI-ALMANAC) <br>
KEGG [Download](https://www.genome.jp/kegg/) <br>
DrugBank [Download](https://go.drugbank.com/) <br>
Cell Model Passports RNA-Seq [Download](https://cog.sanger.ac.uk/cmp/download/rnaseq_20191101.zip) <br>
Cell Model Passports CNV [Download](https://cog.sanger.ac.uk/cmp/download/cnv_20191101.zip) <br>

NOTES: 
We do not know which KEGG dataset has been used here.
The NCI ALMANAC dataset is called the NCI60 in the code which is not the same.


#### Reading our dataset
Now we begin by loading our dataset and editing it into a new file.

In the original code this can be found at `parse/init_parse.py`

Let's do some basic tests and follow their process:

In [50]:
import pandas as pd
import os
import numpy as np

In [51]:
dl_input_df = pd.read_csv('../data-nci/raw_data/nci/nci-raw-data.csv')
dl_input_df.head()

C:\Users\Michael Migacev\AppData\Local\Temp\ipykernel_1384740\1597779311.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  dl_input_df = pd.read_csv('../data-nci/raw_data/nci/nci-raw-data.csv')


,COMBODRUGSEQ,SCREENER,STUDY,TESTDATE,PLATE,PANELNBR,CELLNBR,PREFIX1,NSC1,SAMPLE1,...,PERCENTGROWTH,PERCENTGROWTHNOTZ,TESTVALUE,CONTROLVALUE,TZVALUE,EXPECTEDGROWTH,SCORE,VALID,PANEL,CELLNAME
0,260496,FG,PZUT00156_86_01_T72,06/23/2011,786-0_1_T72,9,18,S,752,37,...,85.979,88.159,332168.0,376781.548,58592.854,90.342,4.0,Y,Renal Cancer,786-0
1,260497,FG,PZUT00156_86_01_T72,06/23/2011,786-0_1_T72,9,18,S,752,37,...,100.903,100.763,379656.0,376781.548,58592.854,87.130,-14.0,Y,Renal Cancer,786-0
2,260498,FG,PZUT00156_86_01_T72,06/23/2011,786-0_1_T72,9,18,S,752,37,...,14.147,27.498,103608.0,376781.548,58592.854,12.739,-1.0,Y,Renal Cancer,786-0
3,260499,FG,PZUT00156_86_01_T72,06/23/2011,786-0_1_T72,9,18,S,752,37,...,71.268,75.736,285360.0,376781.548,58592.854,76.397,5.0,Y,Renal Cancer,786-0
4,260500,FG,PZUT00156_86_01_T72,06/23/2011,786-0_1_T72,9,18,S,752,37,...,89.278,90.945,342664.0,376781.548,58592.854,73.681,-16.0,Y,Renal Cancer,786-0


Now we can copy the first operation from the paper, we can see that given no elements called "Drug A" or "Drug B" we cannot perform the first operation on the raw data.

In [52]:
#dl_input_df = dl_input_df.groupby(['Drug A', 'Drug B', 'Cell Line Name']).agg({'Score':'mean'}).reset_index()
#dl_input_df.head()

To perform the same operations we need to rename the field so they adhere to the raw data. The corresponding fields could be: <br>
`NSC1` or <br>
`SAMPLE1` <br>
This should not matter much as we just want to look at the data for now.

In [53]:
dl_input_df = dl_input_df.groupby(['SAMPLE1', 'SAMPLE2', 'CELLNAME']).agg({'SCORE':'mean'}).reset_index()
print(dl_input_df.shape)
dl_input_df.tail()

(62277, 4)


,SAMPLE1,SAMPLE2,CELLNAME,SCORE
62272,290,31.0,HCT-15,-2.444444
62273,290,46.0,HCT-15,-2.000000
62274,290,52.0,HCT-15,-7.888889
62275,290,91.0,HCT-15,-2.444444
62276,290,203.0,HCT-15,3.222222


In [54]:
dl_input_deletion_list = []
dl_input_df = dl_input_df.fillna('missing')
for row in dl_input_df.itertuples():
    if row[1] == 'missing' or row[2] == 'missing':
        dl_input_deletion_list.append(row[0])
dl_input_df = dl_input_df.drop(dl_input_df.index[dl_input_deletion_list]).reset_index(drop=True)
print(dl_input_df.shape)
dl_input_df.tail()

(62277, 4)


,SAMPLE1,SAMPLE2,CELLNAME,SCORE
62272,290,31.0,HCT-15,-2.444444
62273,290,46.0,HCT-15,-2.0
62274,290,52.0,HCT-15,-7.888889
62275,290,91.0,HCT-15,-2.444444
62276,290,203.0,HCT-15,3.222222


Something seems wrong here. Not only are there some preprocessing operations not mentioned but no missing values are getting removed, this may result in the 2 different datatypes of `SAMPLE1` and `SAMPLE2` <br>
So Let's investigate a bit.

In [78]:
dl_input_df = pd.read_csv('../data-nci/raw_data/nci/nci-raw-data.csv')
dl_input_df.head()

C:\Users\Michael Migacev\AppData\Local\Temp\ipykernel_1384740\1597779311.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  dl_input_df = pd.read_csv('../data-nci/raw_data/nci/nci-raw-data.csv')


,COMBODRUGSEQ,SCREENER,STUDY,TESTDATE,PLATE,PANELNBR,CELLNBR,PREFIX1,NSC1,SAMPLE1,...,PERCENTGROWTH,PERCENTGROWTHNOTZ,TESTVALUE,CONTROLVALUE,TZVALUE,EXPECTEDGROWTH,SCORE,VALID,PANEL,CELLNAME
0,260496,FG,PZUT00156_86_01_T72,06/23/2011,786-0_1_T72,9,18,S,752,37,...,85.979,88.159,332168.0,376781.548,58592.854,90.342,4.0,Y,Renal Cancer,786-0
1,260497,FG,PZUT00156_86_01_T72,06/23/2011,786-0_1_T72,9,18,S,752,37,...,100.903,100.763,379656.0,376781.548,58592.854,87.130,-14.0,Y,Renal Cancer,786-0
2,260498,FG,PZUT00156_86_01_T72,06/23/2011,786-0_1_T72,9,18,S,752,37,...,14.147,27.498,103608.0,376781.548,58592.854,12.739,-1.0,Y,Renal Cancer,786-0
3,260499,FG,PZUT00156_86_01_T72,06/23/2011,786-0_1_T72,9,18,S,752,37,...,71.268,75.736,285360.0,376781.548,58592.854,76.397,5.0,Y,Renal Cancer,786-0
4,260500,FG,PZUT00156_86_01_T72,06/23/2011,786-0_1_T72,9,18,S,752,37,...,89.278,90.945,342664.0,376781.548,58592.854,73.681,-16.0,Y,Renal Cancer,786-0


In [79]:
print(dl_input_df['SAMPLE1'].dtype)
print(dl_input_df['SAMPLE2'].dtype)

int64
float64


I suspect the datatypes have an effect on the missing values, how many missing values do we have when looking at `SAMPLE2`?

In [80]:
print(dl_input_df[['SAMPLE1', 'SAMPLE2', 'CELLNAME', 'SCORE']].isna().sum())

SAMPLE1          0
SAMPLE2     812961
CELLNAME         0
SCORE       815031
dtype: int64


In [81]:
dl_input_df = dl_input_df.dropna()
dl_input_df['SAMPLE2'] = dl_input_df['SAMPLE2'].astype('int64')
print("Missing values in SAMPLE2:", dl_input_df['SAMPLE2'].isna().sum())

Missing values in SAMPLE2: 0


In [82]:
dl_input_df = dl_input_df.groupby(['SAMPLE1', 'SAMPLE2', 'CELLNAME']).agg({'SCORE':'mean'}).reset_index()
print(dl_input_df.shape)
dl_input_df.tail()

(59898, 4)


,SAMPLE1,SAMPLE2,CELLNAME,SCORE
59893,290,31,HCT-15,-2.444444
59894,290,46,HCT-15,-2.000000
59895,290,52,HCT-15,-7.888889
59896,290,91,HCT-15,-2.444444
59897,290,203,HCT-15,3.222222


In [83]:
dl_input_deletion_list = []
dl_input_df = dl_input_df.fillna('missing')
for row in dl_input_df.itertuples():
    if row[1] == 'missing' or row[2] == 'missing':
        dl_input_deletion_list.append(row[0])
dl_input_df = dl_input_df.drop(dl_input_df.index[dl_input_deletion_list]).reset_index(drop=True)
print(dl_input_df.shape)
dl_input_df.tail()

(59898, 4)


,SAMPLE1,SAMPLE2,CELLNAME,SCORE
59893,290,31,HCT-15,-2.444444
59894,290,46,HCT-15,-2.000000
59895,290,52,HCT-15,-7.888889
59896,290,91,HCT-15,-2.444444
59897,290,203,HCT-15,3.222222


## Let's put a stop here

This seems a bit futile without having input from the authors.

## Let's look at the filtered data instead

To do that we copy out the data folders from the DSF repository into our own. <br>
It can be found under `data-nci` don't forget to modify the `.gitignore` so we don't push around much data. <br>

We can ignore the following folders for now: <br>
```
filtered_data-original
plot
result
```

The readme suggests us running the following function to run the model:
```
python geo_tmain_webgnn.py 
```
this file accepts custom specifications for our GNN <br>
More importantly we are "building" our model and using the data here:
```python
def build_geowebgnn_model(args, device, dataset):
    print('--- BUILDING UP WEBGNN MODEL ... ---')
    # GET PARAMETERS
    # [num_gene, num_drug, (adj)node_num]
    final_annotation_gene_df = pd.read_csv('./' + dataset + '/filtered_data/kegg_gene_annotation.csv')
    gene_name_list = list(final_annotation_gene_df['kegg_gene'])
    num_gene = len(gene_name_list)
    drug_num_dict_df = pd.read_csv('./' + dataset + '/filtered_data/drug_num_dict.csv')
    drug_dict = dict(zip(drug_num_dict_df.Drug, drug_num_dict_df.drug_num))
    num_drug = len(drug_dict)
    node_num = num_gene + num_drug
    # [num_gene_edge, num_drug_edge]
    gene_num_df = pd.read_csv('./' + dataset + '/filtered_data/kegg_gene_num_interaction.csv')
    gene_num_df = gene_num_df.drop_duplicates()
    drugbank_num_df = pd.read_csv('./' + dataset + '/filtered_data/final_drugbank_num_sym.csv')
    num_gene_edge = gene_num_df.shape[0]
    num_drug_edge = drugbank_num_df.shape[0]
    num_edge = num_gene_edge + num_drug_edge
    # import pdb; pdb.set_trace()
    # BUILD UP MODEL
    model = WeBGNNDecoder(input_dim=args.input_dim, hidden_dim=args.hidden_dim, embedding_dim=args.output_dim, 
                decoder_dim=args.decoder_dim, node_num=node_num, num_edge=num_edge, num_gene_edge=num_gene_edge, device=device)
    model = model.to(device)
    return model
```
Here we use: <br>
`/filtered_data/kegg_gene_annotation.csv` which is a `.csv` with only 1 column and *2016* gene names <br>
to get the number of the genes <br>
`/filtered_data/drug_num_dict.csv` which is a `.csv` with two columns,<br> 
one the names of the drugs <br>
and the other their number, going from 2017 (continuing from the genes?) to 2034 <br>
so *18* genes <br>
to get the number of the genes.<br>
Then these numbers are added and provided to the `WeBGNNDecoder` as `node_num`<br>
`/filtered_data/kegg_gene_num_interaction.csv` which is a `.csv` with two columns,<br> 
one the number of the gene<br>
the number of the gene it connects to <br>
for 2016 genes these are *18512* connections <br>
to get the number of the connections (2000, 2010 and 2010, 2000 count as 2 connections).<br>
`/filtered_data/final_drugbank_num_sym.csv` which is a `.csv` with two columns,<br> 
one the number of the drug<br>
the number of the gene it connects to <br>
for 2016 genes these are *98* connections <br>
to get the number of the connections (2019, 1 and 1, 2019 count as 2 connections).<br>
Then these numbers are added and provided to the `WeBGNNDecoder` as `edge_num`<br>



#### Quick Check: Are some nodes unconnected?

No

In [13]:
missing_numbers = [
    num for num in range(1, 2017) 
    if not ((gene_num_df['src'] == num).any() or 
            (gene_num_df['dest'] == num).any())
]
print("Missing numbers:", missing_numbers)

Missing numbers: []


#### Quick Check: Are the edges bidirectional?

No, as evident from the code below the num_edges should count edges in both directions seperately, we will see if it is supposed to be this way later. For now I will remebmer this.

In [1]:
import pandas as pd
import numpy as np

In [2]:
gene_num_df = pd.read_csv('../data-nci/filtered_data/kegg_gene_num_interaction.csv')


In [3]:
mirrored_df = gene_num_df.rename(columns={'src':'dest', 'dest':'src'})

# Combine original and mirrored DataFrames
combined = pd.concat([gene_num_df, mirrored_df])

# Find duplicates (including mirrored pairs)
duplicates = combined[combined.duplicated(keep=False)]  # keep=False marks all duplicates

# Show duplicates
print("Duplicate connections (including mirrors):")
print(duplicates.sort_values(by=['src', 'dest']).tail())

# Count duplicates
print(f"\nTotal duplicate pairs: {len(duplicates)//2}")

Duplicate connections (including mirrors):
        src  dest
18356  2007  2000
18470  2008  2000
18357  2008  2000
18488  2010  2000
18358  2010  2000

Total duplicate pairs: 662


In [4]:
gene_num_df.shape

(18512, 2)

In [5]:
gene_num_df_cleaned = gene_num_df.drop_duplicates()
gene_num_df_cleaned.shape

(18512, 2)

A quick check in the `.csv` confirms the result and `drop_duplicates()` does nothing in this dataset.

### Now Let's go to the `WeBGNNDecoder` to see how it is used

It's not used in the `WeBGNNDecoder` at all!!! <br>
Which is what is expected from DNN but I am wondering where the multi-omic data comes in? <br>

#### Moving on we want to find the next place where we use the data

```python
def train_geowebgnn(args, fold_n, load_path, iteration_num, device, dataset):
    # TRAINING DATASET BASIC PARAMETERS
    # [num_feature, num_gene, num_drug]
    num_feature = 4
    dict_drug_num = pd.read_csv('./' + dataset + '/filtered_data/drug_num_dict.csv')
    num_drug = dict_drug_num.shape[0]
    final_annotation_gene_df = pd.read_csv('./' + dataset + '/filtered_data/kegg_gene_annotation.csv')
    num_gene = final_annotation_gene_df.shape[0]
    form_data_path = './' + dataset + '/form_data'
```

It appears to be later in the same file, in our training loop.
Here we use `/form_data` which is not included in the DSF GitHub.

### How is `/form_data` made and what is it?

```python
    xTr = np.load('./' + dataset + '/form_data/xTr' + str(fold_n) + '.npy')
    yTr = np.load('./' + dataset + '/form_data/yTr' + str(fold_n) + '.npy')
    drugTr =  np.load('./' + dataset + '/form_data/drugTr' + str(fold_n) + '.npy')
    edge_index = torch.from_numpy(np.load(form_data_path + '/edge_index.npy')).long() 
```

##### It seems to be our training data!

#### First we Randomize our input

```python
def input_random(dataset):
    final_input_df = pd.read_csv('./' + dataset + '/filtered_data/final_dl_input.csv')
    random_final_input_df = final_input_df.sample(frac = 1)
    random_final_input_df.to_csv('./' + dataset + '/filtered_data/random_final_dl_input.csv', index = False, header = True)
```
This just shuffles our data

#### Then we create our folds

```python
def split_k_fold(k, dataset):
    random_final_dl_input_df = pd.read_csv('./' + dataset + '/filtered_data/random_final_dl_input.csv')
    num_points = random_final_dl_input_df.shape[0]
    num_div = int(num_points / k)
    num_div_list = [i * num_div for i in range(0, k)]
    num_div_list.append(num_points)
    # SPLIT [RandomFinal_NCI60_DeepLearningInput] INTO [k] FOLDS
    for place_num in range(k):
        low_idx = num_div_list[place_num]
        high_idx = num_div_list[place_num + 1]
        print('\n--------TRAIN-TEST SPLIT WITH TEST FROM ' + str(low_idx) + ' TO ' + str(high_idx) + '--------')
        split_input_df = random_final_dl_input_df[low_idx : high_idx]
        split_input_df.to_csv('./' + dataset + '/filtered_data/split_input_' + str(place_num + 1) + '.csv', index = False, header = True)
```
This creates equally big splits as `.csv` files. <br>
#### BUT the `.csv` files from their Repo are faulty
split_1 contains most of the samples and split_2 almost none.<br>
they have it correct in another folder (`filtered_data-original`) but I am not sure which one they used<br>

#### There seems to be no other cross validation except leave drug combination out!







In [9]:
whole_set = pd.read_csv('../data-nci/filtered_data/random_final_dl_input.csv')

In [10]:
whole_set

,Drug A,Drug B,Cell Line Name,Score
0,Dasatinib,Docetaxel,HCT-15,-4.777778
1,Streptozocin,Tretinoin,HOP-62,-12.000000
2,Thalidomide,Paclitaxel,NCI-H322M,1.888889
3,Mitotane,Sirolimus,SN12C,-7.666667
4,Sirolimus,Cladribine,NCI-H322M,-18.555556
...,...,...,...,...
3937,Vorinostat,Nilotinib,DU-145,-4.555556
3938,Paclitaxel,Nilotinib,SF-268,-20.111111
3939,Mitotane,Nilotinib,CCRF-CEM,-8.444444
3940,Everolimus,Romidepsin,SW-620,0.222222


In [12]:
mirrored_whole_set = whole_set.rename(columns={'Drug A':'Drug B', 'Drug B':'Drug A'})

# Combine original and mirrored DataFrames
combined_wh = pd.concat([whole_set, mirrored_whole_set])

# Find duplicates (including mirrored pairs)
duplicates_wh = combined_wh[combined_wh.duplicated(keep=False)]  # keep=False marks all duplicates

# Show duplicates
print("Duplicate connections (including mirrors):")
print(duplicates_wh.sort_values(by=['Drug A', 'Drug B']).tail())

# Count duplicates
print(f"\nTotal duplicate pairs: {len(duplicates_wh)//2}")

Duplicate connections (including mirrors):
Empty DataFrame
Columns: [Drug A, Drug B, Cell Line Name, Score]
Index: []

Total duplicate pairs: 0


No duplicates or Drug A Drug B leakage here!

# CONTINUATION IN `create-data.ipynb`